# tf.metrics
Tensorflow指标算子

## 参考
    性能指标（模型评估）之mAP
    https://blog.csdn.net/u014203453/article/details/77598997
    
## 概念
* Positive:正类
* Negative:负类
* True positive:TP,如果一个实例是正类并且也被 预测成正类，即为真正类(True positive).
* False positive:FP,如果实例是负类被预测成正类，称之为假正类（False positive）.
* True negative:TN,如果实例是负类被预测成负类，称之为真负类（True negative）.
* False negative:FN,正类被预测成负类则为假负类(False negative).
* 混肴矩阵:
   
   1-正，0-反
   
$\begin{array}{c|c|c|c|c}
   真实情况 & 预测为正           & 预测为反  &  合计       & 召回率            \\
   \hline
   正      & TP(真正)           & FN(假反) & TP+FN       & \frac{TP}{TP+FN} \\
   \hline
   反      & FP(假正)           & TN(真反) & FP+TN       &                  \\
   \hline
   合计    & TP+FP              & FN+TN   & TP+FP+FN+TN &                   \\
   \hline
   准确率  & \frac{TP}{TP+FP}   &          &             &                  \\
\end{array}$

* TPR:真正类率(灵敏度) TPR=$\frac{TP}{TP+FN}$ ,分类器所识别出的 正实例占所有正实例的比例
* FPR:假正类率(特异度) FPR=$\frac{FP}{FP+TN}$ ,分类器错认为正类的负实例占所有负实例的比例
* TNR:真负类率        TNR=$\frac{TN}{FP+TN}$ = 1 - FPR。
* Recall:召回率         R=$\frac{TP}{TP+FN}$=TPR
* Precision:准确率      P=$\frac{TP}{TP+FP}$
* ROC:
     - 百度百科:https://baike.baidu.com/item/ROC%E6%9B%B2%E7%BA%BF/775606?fr=aladdin     
     - 受试者工作特征曲线 （receiver operating characteristic curve，简称ROC曲线），又称为感受性曲线（sensitivity curve）。
     - 受试者工作特征曲线就是以假阳性概率FPR（False positive rate）为横轴，真阳性TPR（True positive rate）为纵轴所组成的坐标图，和受试者在特定刺激条件下由于采用不同的判断标准得出的不同结果画出的曲线。
* AUC:
    AUC-AUC的全称是Area under the Curve of ROC，也就是ROC曲线下方的面积。常用来评估二分类模型的训练效果.



In [1]:
import numpy as np
import tensorflow as tf
#创建会话
sess=tf.InteractiveSession()

## 构造测试数据

In [2]:
#第一列为Confidence,#第二列为label
data=np.array([[0.21578134, 1. ],
               [0.28954326, 0. ],
               [0.38596681, 1. ],
               [0.70304854, 1. ],
               [0.36974563, 0. ],
               [0.33001494, 1. ],
               [0.38077377, 0. ],
               [0.20717651, 0. ],
               [0.62806669, 1. ],
               [0.5652305 , 0. ],
               [0.91273938, 1. ],
               [0.25093503, 0. ],
               [0.7101448 , 0. ],
               [0.21007519, 0. ],
               [0.35734559, 0. ],
               [0.02958597, 1. ],
               [0.39668501, 0. ],
               [0.72967364, 1. ],
               [0.33962158, 1. ],
               [0.18520362, 0. ]])
#阈值
threshold_Val=0.5

print(data)


[[0.21578134 1.        ]
 [0.28954326 0.        ]
 [0.38596681 1.        ]
 [0.70304854 1.        ]
 [0.36974563 0.        ]
 [0.33001494 1.        ]
 [0.38077377 0.        ]
 [0.20717651 0.        ]
 [0.62806669 1.        ]
 [0.5652305  0.        ]
 [0.91273938 1.        ]
 [0.25093503 0.        ]
 [0.7101448  0.        ]
 [0.21007519 0.        ]
 [0.35734559 0.        ]
 [0.02958597 1.        ]
 [0.39668501 0.        ]
 [0.72967364 1.        ]
 [0.33962158 1.        ]
 [0.18520362 0.        ]]


 ## 准确度
 tf.metrics.accuracy(
    labels,
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None
)    

In [8]:
import numpy as np
import tensorflow as tf

sess=tf.InteractiveSession()

#构造数据
labels=np.random.randint(0,3,size=(10,1)) #标签值
predicts=np.random.random((10,3))         #预测权重
predicts=np.argmax(predicts,axis=1)       #预测权重转化为标签值
predicts=predicts.reshape((10,))
labels=labels.reshape((10,))
print(labels)
print(predicts)
# Placeholders to take in batches onf data
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.accuracy(tf_label,
                                                  tf_prediction,
                                                  name="metric_accuracy")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_accuracy")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:predicts,tf_label:labels})
acc=sess.run(tf_metric)

print(acc)


[1 0 0 1 1 1 2 0 1 1]
[1 1 2 0 1 2 2 2 2 2]
0.3


# AUC值
    http://baijiahao.baidu.com/s?id=1597939133517926460&wfr=spider&for=pc
    tf.metrics.auc(
        labels,
        predictions,
        weights=None,
        num_thresholds=200,
        metrics_collections=None,
        updates_collections=None,
        curve='ROC',
        name=None,
        summation_method='trapezoidal'
    )

### 参考
    机器学习备忘录 | AUC值的含义与计算方法
    http://baijiahao.baidu.com/s?id=1597939133517926460&wfr=spider&for=pc

In [4]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.auc(tf_label,
                                                  tf_prediction,
                                                  name="metric_auc")

#----------------没有排序的auc值---------------
#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_auc")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
auc=sess.run(tf_metric)

#----------------降序处理的auc值---------------
data_sort=data[np.argsort(data[:,0])[::-1]]
#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_auc")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data_sort[:,0]>threshold_Val,tf_label:data_sort[:,1]})
auc_sort=sess.run(tf_metric)

print('没有排序的auc:',auc)
print('降序处理的auc:',auc_sort)

没有排序的auc: 0.63131315
降序处理的auc: 0.63131315


# 真正
    tf.metrics.true_positives(
        labels,
        predictions,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [15]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.true_positives(tf_label,
                                                   tf_prediction,
                                                   name="metric_tp")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_tp")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
tp=sess.run(tf_metric)

print('TP:',tp)

TP: 4.0


# 假正
    tf.metrics.false_positives(
        labels,
        predictions,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [16]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.false_positives(tf_label,
                                                   tf_prediction,
                                                   name="metric_fp")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_fp")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
fp=sess.run(tf_metric)

print('FP:',fp)

FP: 2.0


# 假反
    tf.metrics.false_negatives(
        labels,
        predictions,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [17]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.false_negatives(tf_label,
                                                   tf_prediction,
                                                   name="metric_fn")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_fn")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
fn=sess.run(tf_metric)

print('FN:',fn)

FN: 5.0


# 真反
    tf.metrics.true_negatives(
        labels,
        predictions,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [18]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.true_negatives(tf_label,
                                                   tf_prediction,
                                                   name="metric_tn")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_tn")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
tn=sess.run(tf_metric)

print('TN:',tn)

TN: 9.0


# TP+FP+FN+TN

In [19]:
print('TP+FP+FN+TN:',tp+fp+fn+tn)

TP+FP+FN+TN: 20.0


# 求均值
    tf.metrics.mean(
        values,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [23]:
# Placeholders
tf_mean = tf.placeholder(dtype=tf.float32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.mean(tf_mean,
                                                   name="metric_mean")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_mean")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_mean:data[:,0]})
mean=sess.run(tf_metric)

print('mean:',mean)

mean: 0.4098679


# 绝对平均误差
    tf.metrics.mean_absolute_error(
        labels,
        predictions,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )


In [29]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.float32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.float32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.mean_absolute_error(tf_label,
                                                   tf_prediction,
                                                   name="metric_mae")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_mae")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0],tf_label:data[:,1]})
mae=sess.run(tf_metric)

print('MAE',mae)

MAE 0.43241802


# mean_IOU
    IOU is defined as follows: IOU = true_positive / (true_positive + false_positive + false_negative).
    tf.metrics.mean_iou(
        labels,
        predictions,
        num_classes,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [ ]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.float32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.float32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.mean_iou .mean_absolute_error(tf_label,
                                                   tf_prediction,
                                                   name="metric_mae")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_mae")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0],tf_label:data[:,1]})
mae=sess.run(tf_metric)

print('MAE',mae)

# 余弦距离--有疑问？
    tf.metrics.mean_cosine_distance(
        labels,
        predictions,
        dim,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [32]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.float32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.float32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.mean_cosine_distance(tf_label,
                                                   tf_prediction,
                                                   0,
                                                   name="metric_mcd")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_mcd")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update,feed_dict={tf_prediction:data[:,0],tf_label:data[:,1]})
mcd=sess.run(tf_metric)

print('MCD',mcd)

MCD -3.274499


# 真正(带阈值)--有疑问？
    tf.metrics.true_positives_at_thresholds(
        labels,
        predictions,
        thresholds,
        weights=None,
        metrics_collections=None,
        updates_collections=None,
        name=None
    )

In [9]:
# Placeholders
tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations
tf_metric, tf_metric_update = tf.metrics.true_positives_at_thresholds(tf_label,
                                                   tf_prediction,
                                                   [0.05],
                                                   name="metric_tp_at_t")

#初始化变量
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metric_tp_at_t")
# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

#初始化全局变量
sess.run(tf.global_variables_initializer())
# initialize/reset the running variables
sess.run(running_vars_initializer)
sess.run(tf_metric_update
         ,feed_dict={tf_prediction:data[:,0]>threshold_Val,tf_label:data[:,1]})
tp=sess.run(tf_metric)

print(data[:,0]>threshold_Val)
print(data[:,1]>0)
print('TP:',tp)

[False False False  True False False False False  True  True  True False
  True False False False False  True False False]
[ True False  True  True False  True False False  True False  True False
 False False False  True False  True  True False]
TP: [4.]
